1. Check if Video is 360p, if not convert to 360p
2. Use psv files and json to only target sections of podcast where guest is speaking
    (do not want biased joe rogan data)
3. Extract 4 second clips at 10fps gray scale gifs
4. Filter out any clips that have a 'scene change' aka the camera pans to the listener

In [1]:
#Global variables
import socket
import os
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data\\'
ENV_FOLDER_DATA_PROC = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data_Proc\\'
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

ENV_HOSTNAME:The-Beast


In [2]:
#!pip install ffmpeg-python

import ffmpeg

input_file = ENV_FOLDER_DATA  + 'Joe Rogan Experience #568 - Dr. Rhonda Patrick.mp4'

def get_video_resolution(video_file):
    probe = ffmpeg.probe(video_file)
    video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
    if video_stream:
        width = int(video_stream['width'])
        height = int(video_stream['height'])
        resolution = (width, height)
        return resolution
    else:
        return None

def is_360p(video_file):
    resolution = get_video_resolution(video_file)
    if resolution:
        width, height = resolution
        if width <= 640 and height <= 360:
            return True
    return False

def convert_to_360p(video_file, output_file):
    ffmpeg.input(video_file).output(output_file, s='640x360').run()


In [3]:
# Load Json Speakers

import json

file = ENV_FOLDER_DATA + "..\\example files\\speakers.json"

ep_speakers = json.load(open(file))

# for ep in ep_speakers:
#     speakers = ep_speakers[ep]
#     for speaker_id,  speaker_val in speakers.items():
#         print(speaker_id, speaker_val)

In [4]:
import csv

# def extract_speaker_times(data):
#     total_start_time = None
#     total_end_time = None
#     current_speaker = None
#     speaker_times = []

#     for entry in data:
#         _, speaker, start_time, end_time = entry

#         if current_speaker is None:
#             current_speaker = speaker
#             total_start_time = float(start_time)
#             total_end_time = float(end_time)
#         elif current_speaker == speaker:
#             total_end_time = float(end_time)
#         else:
#             speaker_times.append((current_speaker, total_start_time, total_end_time))
#             current_speaker = speaker
#             total_start_time = float(start_time)
#             total_end_time = float(end_time)

#     # Add the last speaker's total speaking time
#     if current_speaker is not None:
#         speaker_times.append((current_speaker, total_start_time, total_end_time))

#     return speaker_times

# psv_file = ENV_FOLDER_DATA_PROC + "segmentation\\501.psv"

# with open(psv_file, 'r') as psv_file:
#     psv_data = list(csv.reader(psv_file, delimiter='|'))

# print(extract_speaker_times(psv_data))

In [5]:
import csv
import json

def filter_rows(speaker_times, json_dict_ep):

    
    # Extract speakers to ignore
    ignore_speakers = ['Joe Rogan', 'Young Jaime', 'Misc']
    allowed_speaker_id = []
    for speaker_id, speaker_val in json_dict_ep.items():
        if speaker_val not in ignore_speakers:
            allowed_speaker_id.append(speaker_id)
    
    # Filter rows
    filtered_rows = []
    for row in speaker_times:
        speaker_id = row[1]
        if speaker_id in allowed_speaker_id:
            filtered_rows.append(row)
    
    return filtered_rows


# filtered_rows = filter_rows(psv_data, ep_speakers['501'])
# for i in filtered_rows:
#     print(i)
# print(len(filtered_rows))

In [6]:
# def extract_clip_times(filtered_rows):
#     clip_times = []
#     for i in range(len(filtered_rows) - 1):
#         _, speaker, start_time, _ = filtered_rows[i]
#         _, _, _, end_time = filtered_rows[i + 1]
#         clip_times.append((speaker, float(start_time), float(end_time)))
#     return clip_times

# four_sec_segs = extract_4sec_segs(filtered_rows)
# for i in four_sec_segs:
#     print(i)

# print(len(four_sec_segs))

In [7]:
# create gif in 10fps grayscale from mp4 and 4 second segments

import ffmpeg
import os

def create_clips(video_file, filtered_rows, episode_number):

    #create folder for episode
    folder = ENV_FOLDER_DATA_PROC + "clips\\" + str(episode_number)

    if not os.path.exists(folder):
        os.makedirs(folder)

    for seg, _, start, end in filtered_rows:
        start = round(float(start),1)
        end = round(float(end),1)

        if end - start > 1:
            output = folder + "\\{}_seg{}.mp4".format(episode_number,seg)
            ffmpeg.input(video_file, ss=start, t=min(4,end-start)).output(output, vf='fps=10,format=gray').run()

# video_file = ENV_FOLDER_DATA + "Joe Rogan Experience #501 - Randall Carlson.mp4"
# create_clips(video_file, four_sec_segs, 501)

In [8]:
import cv2
import glob

def detect_scene_changes(clip_path, threshold=10):
    clip = cv2.VideoCapture(clip_path)
    frame_rate = clip.get(cv2.CAP_PROP_FPS) # Get frame rate
    prev_frame = None
    scene_changes = []

    frame_number = 0
    while True:
        ret, frame = clip.read()
        if not ret:
            break
        
        
        if prev_frame is not None:
            # Calculate frame difference
            frame_diff = cv2.absdiff(frame, prev_frame)
            diff_score = frame_diff.mean()
            
            # Check if scene change detected
            if diff_score > threshold:
                # Calculate time in seconds
                time_in_seconds = frame_number / frame_rate
                scene_changes.append((time_in_seconds, diff_score))
        
        prev_frame = frame
        frame_number += 1

    return scene_changes

# Example usage

# for file in glob.glob(ENV_FOLDER_DATA_PROC + 'gifs\\501\\*.gif'):
#     scene_changes = detect_scene_changes(file)
#     print("Scene changes for", file)
#     for change in scene_changes:
#         print("Time:", change[0], "seconds, Score:", change[1])




In [9]:

def delete_clips_with_scene_changes(gif_path, threshold=10):

    for file in glob.glob(gif_path):
        scene_changes = detect_scene_changes(file)
        if len(scene_changes) == 0:
            continue

        #print("Deleting", file)
        os.remove(file)

# Example usage
# delete_clips_with_scene_changes(ENV_FOLDER_DATA_PROC + "clips\\501\\*.mp4")

In [10]:
# #open pickle file

# import pickle

# pickle_file = ENV_FOLDER_DATA_PROC + "targetdf\\501.pickle"

# with open(pickle_file, 'rb') as f:
#     target_df = pickle.load(f)

# #print(target_df)

# for row in target_df.iterrows():
#     for seg in row:
#         print(seg)

In [11]:

import numpy as np

def preprocess_video(video_path, desired_fps=10, target_resolution=(640, 360)):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, target_resolution)
        frames.append(frame)

    cap.release()

    frame_interval = int(10 / desired_fps)
    sampled_frames = frames[::frame_interval]

    video_data = np.array(sampled_frames)
    video_data = video_data / 255.0

    video_data = np.expand_dims(video_data, axis=0)
    video_data = np.expand_dims(video_data, axis=-1)

    return video_data

# Example usage:
# video_path = ENV_FOLDER_DATA_PROC + 'clips\\501\\501_524_528.mp4'
# preprocessed_data = preprocess_video(video_path)
# print("Preprocessed video data shape:", preprocessed_data.shape)

In [12]:

def split_vids(Data_folder):

    files = glob.glob(Data_folder + '*.mp4')
    
    #reverse order to process the last file first
    files.sort(reverse=True)

    for file in files:
    
        

        episode_number = file.split('\\')[-1].split('#')[-1].split(' ')[0]
        
        #if clip folder already exists, skip
        if os.path.exists(ENV_FOLDER_DATA_PROC + "clips\\" + episode_number):
            print("Skipping", episode_number)
            continue
        else:
            print("Processing", episode_number)
        
        vid = file
        if not is_360p(vid):
            print("Converting", vid, "to 360p")
            convert_to_360p(vid, vid)
        
        psv_file = ENV_FOLDER_DATA_PROC + "segmentation\\" + episode_number + ".psv"


        with open(psv_file, 'r') as psv_file:
            psv_data = list(csv.reader(psv_file, delimiter='|'))

        if not episode_number in ep_speakers:
            continue


        filtered_rows = filter_rows(psv_data, ep_speakers[episode_number]) 

        create_clips(vid, filtered_rows, episode_number)

        delete_clips_with_scene_changes(ENV_FOLDER_DATA_PROC + "clips\\" + episode_number + "\\*.mp4")

        print("Finished processing", episode_number)

split_vids(ENV_FOLDER_DATA)

Skipping 595
Skipping 594
Skipping 593
Skipping 592
Skipping 591
Skipping 590
Skipping 589
Skipping 588
Skipping 587
Skipping 585
Skipping 584
Skipping 583
Skipping 582
Skipping 581
Skipping 579
Skipping 578
Skipping 577
Skipping 576
Skipping 575
Skipping 573
Skipping 571
Skipping 570
Skipping 569
Skipping 568
Skipping 567
Skipping 566
Skipping 565
Skipping 564
Skipping 563
Skipping 562
Skipping 561
Skipping 560
Skipping 559
Skipping 558
Skipping 557
Skipping 556
Skipping 555
Skipping 554
Skipping 553
Skipping 552
Skipping 551
Skipping 550
Skipping 549
Skipping 548
Skipping 547
Skipping 546
Skipping 545
Skipping 544
Skipping 543
Skipping 542
Skipping 541
Skipping 540
Skipping 539
Skipping 538
Skipping 537
Skipping 536
Skipping 535
Skipping 534
Skipping 533
Skipping 532
Skipping 531
Skipping 529
Skipping 527
Skipping 526
Skipping 525
Skipping 523
Skipping 522
Skipping 520
Skipping 519
Skipping 518
Processing 515
Skipping 514
Skipping 513
Skipping 512
Skipping 511
Processing 510
Skipping